# Find precipitation value for pulsed and upslope storms

In [1]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')

import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')



In [2]:
m = ['11', '12', '01', '02', '03']
station = 'Haukeliseter'

In [3]:
savefig = 0
if savefig == 1:
    figdir = '../../Figures/Weathermast_MEPS_Retrieval/%s/precip_seasonal_stat' %(station)
    cF.createFolder('%s/' %figdir)
form = 'png'

### Minuelty data

In [4]:
obs_f = dict()
obs_double_fence_acc_mean = dict()
obs_wind_dir_10m   = dict()
obs_wind_speed_10m = dict()
obs_dofe_acc   = dict()
obs_wind_dir   = dict()
obs_wind_speed = dict() 
obs_dofe_acc_rolling = dict()
obs_wind_dir_rolling = dict()
obs_wind_speed_rolling = dict()
obs_diff = dict()
obs_westerly_idx = dict()
obs_easterly_idx = dict()
obs_westerly = dict()
obs_easterly = dict()
obs_total_precip = dict()
obs_precip_west  = dict()
obs_precip_east  = dict()
obs_double_fence_temp = dict()
obs_air_temp = dict()
obs_air_temp_rolling = dict()

obs_diff_2deg = dict()
obs_total_precip_2deg = dict()
obs_precip_east_2deg = dict()
obs_precip_west_2deg = dict()

for month in m:
    if month == '11':
        t = np.arange(7,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        obs_dirnc = '../../Data/Weathermast/Met_%s%s%s_%s.nc' %(year,month,day,station)
        obs_f[int(year+month+day)] = netCDF4.Dataset(obs_dirnc, 'r')

        obs_double_fence_acc_mean[int(year+month+day)] = (np.nanmean(obs_f[int(year+month+day)].variables['double_fence_accum'][:], axis=1))
        obs_wind_dir_10m[int(year+month+day)]   = (obs_f[int(year+month+day)].variables['10m_wind_dir_mast1'][:])
        obs_wind_speed_10m[int(year+month+day)] = (obs_f[int(year+month+day)].variables['10m_wind_speed_mast1'][:])
        obs_double_fence_temp[int(year+month+day)] = (obs_f[int(year+month+day)].variables['air_temp'][:])
        
        obs_f[int(year+month+day)].close
        
        ## Create a rolling mean to smoothen out the data
        # attach the last 9 minutes from the previous day to calculate the rolling mean every 10 minutes
        if month != '11' and day == '01':
            if month == '12':
                previous_day = '30'
            elif month == '01' or month == '02':
                previous_day = '31'
            elif month == '03':
                previous_day = '28'
            if month == '01':
                previous_month = '12'
                previous_year = str(int(year)-1)
            elif month == '12':
                previous_month = str(int(month)-1)
                previous_year = year
            else:
                previous_month = '0%s' %(int(month)-1)
                previous_year = year
            
        elif int(day)-1 < 10:
            previous_day = '0%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        else:
            previous_day = '%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        if year == '2016' and month == '11' and day == '07':
            continue
        else:
            pp = (np.concatenate((obs_double_fence_acc_mean[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_acc_mean[int(year+month+day)][:])))
            obs_dofe_acc[int(year+month+day)] = (pp)
            wd = (np.concatenate((obs_wind_dir_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_dir_10m[int(year+month+day)][:])))
            obs_wind_dir[int(year+month+day)] = (wd)
            ws = (np.concatenate((obs_wind_speed_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_speed_10m[int(year+month+day)][:])))
            obs_wind_speed[int(year+month+day)] = (ws)
            at = (np.concatenate((obs_double_fence_temp[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_temp[int(year+month+day)][:])))
            obs_air_temp[int(year+month+day)] = at
            
        # Calculate the moving average. That is, take
        # the first ten values, average them, 
        # then drop the first and add the eleventh, etc.
        obs_dofe_acc[int(year+month+day)] = pd.DataFrame(np.array(obs_dofe_acc[int(year+month+day)])[:])
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc[int(year+month+day)].rolling(window=10,
                                                                                              min_periods= 7,
                                                                                              axis=0).mean()
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc_rolling[int(year+month+day)].values
        obs_dofe_acc_rolling[int(year+month+day)] = np.array(obs_dofe_acc_rolling[int(year+month+day)])[9:,:]

        obs_wind_dir[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_dir[int(year+month+day)])[:])
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir[int(year+month+day)].rolling(window=10,
                                                                                              min_periods=7,
                                                                                              axis=0).mean()
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir_rolling[int(year+month+day)].values
        obs_wind_dir_rolling[int(year+month+day)] = np.array(obs_wind_dir_rolling[int(year+month+day)])[9:,:]

        obs_wind_speed[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_speed[int(year+month+day)])[:])
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed[int(year+month+day)].rolling(window=10,
                                                                                                  min_periods=7,
                                                                                                  axis=0).mean()
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed_rolling[int(year+month+day)].values
        obs_wind_speed_rolling[int(year+month+day)] = np.array(obs_wind_speed_rolling[int(year+month+day)])[9:,:]
        
        obs_air_temp[int(year+month+day)] = pd.DataFrame(np.array(obs_air_temp[int(year+month+day)])[:])
        obs_air_temp_rolling[int(year+month+day)] = obs_air_temp[int(year+month+day)].rolling(window=10,
                                                                                                  min_periods=7,
                                                                                                  axis=0).mean()
        obs_air_temp_rolling[int(year+month+day)] = obs_air_temp_rolling[int(year+month+day)].values
        obs_air_temp_rolling[int(year+month+day)] = np.array(obs_air_temp_rolling[int(year+month+day)])[9:,:]
        
        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        obs_diff[int(year+month+day)] = np.empty(shape=(obs_dofe_acc_rolling[int(year+month+day)].shape))#/60))
        obs_diff[int(year+month+day)][:] =  np.nan

        for i in np.arange(0,obs_dofe_acc_rolling[int(year+month+day)].shape[0]-1):#/60):
            obs_diff[int(year+month+day)][int(i),:] = obs_dofe_acc_rolling[int(year+month+day)][int(i)+1] - \
                                                      obs_dofe_acc_rolling[int(year+month+day)][int(i)]


        # Find wind regimes and associated precip amount
        IDX = np.arange(obs_wind_dir_rolling[int(year+month+day)].shape[0])

        ## find all obs_westerly regimes (202.5 - 22.5 deg) 
        obs_westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 202.5), 
                                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <= 360)),
                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <22.5))]
        ## find all obs_easterly regimes (22.5 - 202.5 deg)
        obs_easterly_idx[int(year+month+day)]  = IDX[np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 22.5),
                                                                (obs_wind_dir_rolling[int(year+month+day)][:,0] < 202.5))]

        ## find indices for all obs_westerly regimes 
        sub_idx = []

        for i in range(0,obs_westerly_idx[int(year+month+day)] .shape[0]-1):
            if obs_westerly_idx[int(year+month+day)] [i]+1 == obs_westerly_idx[int(year+month+day)] [i+1]:
                continue
            else:
                #print(i)
                sub_idx.append(i)
    
    #        print([int(year+month+day)], obs_westerly_idx[int(year+month+day)], sub_idx)
        
        
        ## find westerlies which are < 29 min and assign them to obs_easterly wind regime
        if len(obs_westerly_idx[int(year+month+day)]) == 0 and len(sub_idx) == 0:     # westerlies do not exist
            obs_westerly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]
#            print('No obs_westerly wind on %s' %(int(year+month+day)))
        elif len(obs_westerly_idx[int(year+month+day)]) != 0 and len(sub_idx) == 0:   # westerlies exist only for one time period a day
            obs_westerly[int(year+month+day)] = []
      #      obs_easterly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]
            
            obs_westerly_duration = obs_westerly_idx[int(year+month+day)][-1] - obs_westerly_idx[int(year+month+day)][0]
            if obs_westerly_duration >= 28:
                obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)],
                                                                obs_westerly_idx[int(year+month+day)][:]))
            elif obs_westerly_duration < 28:
                obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                obs_westerly_idx[int(year+month+day)][:]))
        else:
            obs_westerly[int(year+month+day)] = []
         #   obs_easterly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]

            for i in np.arange(0,np.array(sub_idx).shape[0]+1):                # westerlies and easterlies interchange daily
                if i == 0:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [ sub_idx[0]] - \
                                         obs_westerly_idx[int(year+month+day)] [0])
                elif i == np.array(sub_idx).shape[0]:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [-1] - \
                                         obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1])
                else:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [ sub_idx[i]] - \
                                         obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1])
                if obs_westerly_duration >= 28:
                    if i == 0:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
            #            print(obs_westerly_duration, RR_obs_westerly)
                    elif i == np.array(sub_idx).shape[0]:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
             #           print(obs_westerly_duration, RR_obs_westerly)
                    else:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
              #          print(obs_westerly_duration, RR_obs_westerly)
                elif obs_westerly_duration < 28:
                    if i == 0:
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                   # print(obs_westerly_duration)
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
        

        if len(obs_westerly[int(year+month+day)]) == 0:    # only easterlies
            obs_easterly[int(year+month+day)]  = obs_easterly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
   #                        np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:]),2 ),
    #              'obs_easterly [mm/24h]:', 
     #             np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:]),2),
      #            'obs_westerly [mm/24h]:', 
       #           np.round(len(obs_westerly[int(year+month+day)]),2))
        elif len(obs_easterly[int(year+month+day)]) == 0:  # only westerlies
            obs_westerly[int(year+month+day)]  = obs_westerly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
   #                        np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:]),2 ),
    #              'obs_easterly [mm/24h]:', 
     #             np.round(len(obs_easterly[int(year+month+day)]),2),
      #            'obs_westerly [mm/24h]:', 
       #           np.round(np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:]),2))
        else:    # both easterlies and westerlies
            obs_easterly[int(year+month+day)]  = obs_easterly[int(year+month+day)].astype(int)
            obs_westerly[int(year+month+day)]  = obs_westerly[int(year+month+day)].astype(int)
            
#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
   #                        np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:]),2 ),
    #              'obs_easterly [mm/24h]:', 
     #             np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:]),2),
      #            'obs_westerly [mm/24h]:', 
       #           np.round(np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:]),2))
            
        # assign sum values to days
        if len(obs_easterly[int(year+month+day)]) == 0 and len(obs_westerly[int(year+month+day)]) == 0:
            obs_total_precip[int(year+month+day)] = np.nan
            obs_precip_east[int(year+month+day)] =  np.nan
            obs_precip_west[int(year+month+day)] =  np.nan
        else:
            obs_total_precip[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
                                                np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:])
            obs_precip_east[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:])
            obs_precip_west[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:])
            
#        print([int(year+month+day)], 'total:', np.round(obs_total_precip[int(year+month+day)],2),
 #                                       'westerly:',  np.round(obs_precip_west[int(year+month+day)],2),
  #                                       'easterly:', np.round(obs_precip_east[int(year+month+day)],2) )
        
        
        #### assign nan values where air temperature is larger than 2 degC
        obs_diff_2deg[int(year+month+day)] = obs_diff[int(year+month+day)][:]
        obs_diff_2deg[int(year+month+day)][np.where(obs_air_temp_rolling[int(year+month+day)] > 2.),:] = np.nan
        # assign sum values to days with occuring temp < 2degC
        if len(obs_easterly[int(year+month+day)]) == 0 and len(obs_westerly[int(year+month+day)]) == 0:
            obs_total_precip_2deg[int(year+month+day)] = np.nan
            obs_precip_east_2deg[int(year+month+day)] =  np.nan
            obs_precip_west_2deg[int(year+month+day)] =  np.nan
        else:
            obs_total_precip_2deg[int(year+month+day)] = np.nansum(obs_diff_2deg[int(year+month+day)][obs_easterly[int(year+month+day)],:]) + \
                                                         np.nansum(obs_diff_2deg[int(year+month+day)][obs_westerly[int(year+month+day)],:])
            obs_precip_east_2deg[int(year+month+day)] = np.nansum(obs_diff_2deg[int(year+month+day)][obs_easterly[int(year+month+day)],:])
            obs_precip_west_2deg[int(year+month+day)] = np.nansum(obs_diff_2deg[int(year+month+day)][obs_westerly[int(year+month+day)],:])
        

### Eklima data

In [5]:
eklima_f = dict()
eklima_double_fence_acc = dict()
eklima_wind_dir_10m   = dict()
eklima_wind_speed_10m = dict()
eklima_diff = dict()
eklima_westerly_idx = dict()
eklima_easterly_idx = dict()
eklima_westerly = dict()
eklima_easterly = dict()
eklima_total_precip = dict()
eklima_precip_west  = dict()
eklima_precip_east  = dict()
eklima_air_temp = dict()

eklima_diff_2deg = dict()
eklima_total_precip_2deg = dict()
eklima_precip_east_2deg = dict()
eklima_precip_west_2deg = dict()

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        eklima_dirnc = '../../Data/Eklima/%s/precip_wind_temp_%s%s%s.nc' %(station,year,month,day)
        eklima_f[int(year+month+day)] = netCDF4.Dataset(eklima_dirnc, 'r')

        eklima_double_fence_acc[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['RR_1'][:])
        eklima_wind_dir_10m[int(year+month+day)]   = (eklima_f[int(year+month+day)].variables['DD'][:])
        eklima_wind_speed_10m[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['FF'][:])
        eklima_air_temp[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['TA'][:])
        
        eklima_f[int(year+month+day)].close
        

        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        eklima_diff[int(year+month+day)] = np.empty(shape=(eklima_double_fence_acc[int(year+month+day)].shape))#/60))
        eklima_diff[int(year+month+day)][:] =  np.nan

        eklima_diff[int(year+month+day)][:,] = eklima_double_fence_acc[int(year+month+day)]

        # Find wind regimes and associated precip amount
        IDX = np.arange(eklima_wind_dir_10m[int(year+month+day)].shape[0])

        ## find all eklima_westerly regimes (202.5 - 22.5 deg) 
        eklima_westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 202.5),
                                                                                     (eklima_wind_dir_10m[int(year+month+day)][:,] <= 360)),
                                                                      (eklima_wind_dir_10m[int(year+month+day)][:,] <22.5))]

        ## find all eklima_easterly regimes (22.5 - 202.5 deg)
        eklima_easterly_idx[int(year+month+day)]  = IDX[np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 22.5),
                                                                       (eklima_wind_dir_10m[int(year+month+day)][:,] < 202.5))]

        eklima_westerly[int(year+month+day)] = eklima_westerly_idx[int(year+month+day)]
        eklima_easterly[int(year+month+day)] = eklima_easterly_idx[int(year+month+day)]

        if len(eklima_westerly[int(year+month+day)]) == 0:    # only easterlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(len(eklima_westerly[int(year+month+day)]),2))

        elif len(eklima_easterly[int(year+month+day)]) == 0:  # only westerlies
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(len(eklima_easterly[int(year+month+day)]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],]),2))
        else:    # both easterlies and westerlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)
            
#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],]),2))

        # assign sum values to days
        if len(eklima_easterly[int(year+month+day)]) == 0 and len(eklima_westerly[int(year+month+day)]) == 0:
            eklima_total_precip[int(year+month+day)] = np.nan
            eklima_precip_east[int(year+month+day)] = np.nan
            eklima_precip_west[int(year+month+day)] = np.nan
        else:
            eklima_total_precip[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
                                           np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],])
            eklima_precip_east[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],])
            eklima_precip_west[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],])
        

        #### assign nan values where air temperature is larger than 2 degC
        eklima_diff_2deg[int(year+month+day)] = eklima_diff[int(year+month+day)][:]
        eklima_diff_2deg[int(year+month+day)][np.where(eklima_air_temp[int(year+month+day)] > 2.),] = np.nan
        # assign sum values to days with occuring temp < 2degC
        if len(eklima_easterly[int(year+month+day)]) == 0 and len(eklima_westerly[int(year+month+day)]) == 0:
            eklima_total_precip_2deg[int(year+month+day)] = np.nan
            eklima_precip_east_2deg[int(year+month+day)] =  np.nan
            eklima_precip_west_2deg[int(year+month+day)] =  np.nan
        else:
            eklima_total_precip_2deg[int(year+month+day)] = np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_easterly[int(year+month+day)],]) + \
                                                         np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_westerly[int(year+month+day)],])
            eklima_precip_east_2deg[int(year+month+day)] = np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_easterly[int(year+month+day)],])
            eklima_precip_west_2deg[int(year+month+day)] = np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_westerly[int(year+month+day)],])
        

### assign eklima data where total precip < 0
        if obs_total_precip[int(year+month+day)] < 0:
            if obs_precip_east[int(year+month+day)] < 0 and obs_precip_west[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
    
            elif obs_precip_east[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
        
            elif obs_precip_west[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
        
        elif obs_precip_east[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])   
    
        elif obs_precip_west[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
        elif np.isnan(obs_total_precip[int(year+month+day)]):
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
        
#        print([int(year+month+day)], 'observ total:', np.round(obs_total_precip[int(year+month+day)],2),
 #                                       'westerly:',  np.round(obs_precip_west[int(year+month+day)],2),
  #                                       'easterly:', np.round(obs_precip_east[int(year+month+day)],2) )

### MEPS data

In [6]:
def fill_values(variable):
    if np.ma.is_masked(variable):
        mask = np.ma.getmaskarray(variable[:,:,:])
        marr = np.ma.array(variable[:,:,:], mask = mask, fill_value = np.nan)
        filled = marr.filled(np.nan)
        
    else:
        filled = variable[:]
        
    return(filled)

In [7]:
meps_f = dict()
meps_precip_acc = dict()
meps_xwind_10m = dict()
meps_ywind_10m = dict()
meps_diff = dict()
meps_wind_dir   = dict()
meps_wind_speed = dict()
meps_westerly_idx = dict()
meps_easterly_idx = dict()
meps_westerly = dict()
meps_easterly = dict()
meps_total_precip = dict()
meps_precip_west  = dict()
meps_precip_east  = dict()
meps_air_temperature_2m = dict()

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        meps_dirnc = '../../Data/MEPS/%s/%s%s/sfc_hybrid5_allEM_00/%s%s%s_00.nc' %(station,year,month,year,month,day)
        try:
            meps_f[int(year+month+day)] = netCDF4.Dataset(meps_dirnc, 'r')
        except OSError:
            print('no file found: %s' %meps_dirnc)
#            meps_precip_acc[int(year+month+day)] = np.empty(shape=(67, 1, 10))
 #           meps_xwind_10m[int(year+month+day)] = np.empty(shape=(67, 1, 10))
  #          meps_ywind_10m[int(year+month+day)] = np.empty(shape=(67, 1, 10))
            
            meps_total_precip[int(year+month+day)] = np.empty(shape=(10, ))
            meps_total_precip[int(year+month+day)][:] = np.nan
            meps_precip_east[int(year+month+day)] = np.empty(shape=(10, ))
            meps_precip_east[int(year+month+day)][:] = np.nan
            meps_precip_west[int(year+month+day)] = np.empty(shape=(10, ))
            meps_precip_west[int(year+month+day)][:] = np.nan
            continue

        meps_precip_acc[int(year+month+day)] = (meps_f[int(year+month+day)].variables['precipitation_amount_acc'][:])
        meps_xwind_10m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['x_wind_10m'][:])
        meps_ywind_10m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['y_wind_10m'][:])
        meps_air_temperature_2m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['air_temperature_2m'][:])
        
        ## substitute missing values with nan
        meps_precip_acc[int(year+month+day)] = fill_values(meps_precip_acc[int(year+month+day)][:,:,:])
        meps_xwind_10m[int(year+month+day)] = fill_values(meps_xwind_10m[int(year+month+day)][:,:,:])
        meps_ywind_10m[int(year+month+day)] = fill_values(meps_ywind_10m[int(year+month+day)][:,:,:])
        meps_air_temperature_2m[int(year+month+day)] = fill_values(meps_air_temperature_2m[int(year+month+day)][:,:,:])
        
#        meps_wind_dir_10m[int(year+month+day)]   = (meps_f[int(year+month+day)].variables['10m_wind_dir_mast1'][:])
#        meps_wind_speed_10m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['10m_wind_speed_mast1'][:])
        
        meps_f[int(year+month+day)].close
        

        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        meps_diff[int(year+month+day)] = np.empty(shape=(meps_precip_acc[int(year+month+day)].shape))#/60))
        meps_diff[int(year+month+day)][:] =  np.nan

        for i in np.arange(0,meps_precip_acc[int(year+month+day)].shape[0]-1):
            meps_diff[int(year+month+day)][int(i),:,:] = meps_precip_acc[int(year+month+day)][int(i)+1,:,:] - \
                                                         meps_precip_acc[int(year+month+day)][int(i),:,:]

        #### calculate wind direction / speed
        meps_wind_dir[int(year+month+day)] = np.arctan2(meps_ywind_10m[int(year+month+day)][:,:,:], 
                                                        meps_xwind_10m[int(year+month+day)][:,:,:]) 
        meps_wind_dir[int(year+month+day)] = 270 - np.rad2deg(meps_wind_dir[int(year+month+day)])
        larger360 = np.where(meps_wind_dir[int(year+month+day)][:,:,:] > 360.)
        meps_wind_dir[int(year+month+day)][larger360] = meps_wind_dir[int(year+month+day)][larger360]-360
    
        meps_wind_speed[int(year+month+day)] = np.sqrt((meps_xwind_10m[int(year+month+day)][:,:,:])**2 + \
                                                           (meps_ywind_10m[int(year+month+day)][:,:,:])**2 )
        
        ### only get values up to 24h
        meps_diff[int(year+month+day)] = meps_diff[int(year+month+day)][:25,:,:]
        meps_wind_dir[int(year+month+day)] = meps_wind_dir[int(year+month+day)][:25,:,:]
        meps_wind_speed[int(year+month+day)] = meps_wind_speed[int(year+month+day)][:25,:,:]
        meps_air_temperature_2m[int(year+month+day)] = meps_air_temperature_2m[int(year+month+day)][:25,:,:]

        # Find wind regimes and associated precip amount
        IDX = np.arange(meps_wind_dir[int(year+month+day)].shape[0])

        meps_westerly_idx[int(year+month+day)] = []
        meps_easterly_idx[int(year+month+day)] = []

        for ens_memb in range(10):
            ## find all meps_westerly regimes (202.5 - 22.5 deg) 
            meps_westerly_idx[int(year+month+day)].append(IDX[np.logical_or(
                    np.logical_and((meps_wind_dir[int(year+month+day)][:,0,ens_memb] >= 202.5), 
                                   (meps_wind_dir[int(year+month+day)][:,0,ens_memb] <= 360)),
                    (meps_wind_dir[int(year+month+day)][:,0,ens_memb] <22.5))])
#        print(np.array(meps_westerly_idx[int(year+month+day)][ens_memb]).shape)


            ## find all meps_easterly regimes (22.5 - 202.5 deg)
            meps_easterly_idx[int(year+month+day)].append(IDX[np.logical_and(
                    (meps_wind_dir[int(year+month+day)][:,0,ens_memb] >= 22.5),
                    (meps_wind_dir[int(year+month+day)][:,0,ens_memb] < 202.5))])
#        print(np.array(meps_easterly_idx[int(year+month+day)][ens_memb]).shape)

        
        meps_westerly[int(year+month+day)] = meps_westerly_idx[int(year+month+day)]
        meps_easterly[int(year+month+day)] = meps_easterly_idx[int(year+month+day)]
        
        meps_easterly_idx[int(year+month+day)] = np.array(meps_easterly_idx[int(year+month+day)])
        meps_westerly_idx[int(year+month+day)] = np.array(meps_westerly_idx[int(year+month+day)])

        meps_total_precip[int(year+month+day)] = []
        meps_precip_east[int(year+month+day)] = []
        meps_precip_west[int(year+month+day)] = []

        for ens_memb in range(10):
#            if len(meps_westerly[int(year+month+day)][ens_memb]) == 0:    # only easterlies
#                print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 'EM=%s' %ens_memb,
  #                np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
   #                        np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2 ),
    #              'meps_easterly [mm/24h]:', 
     #             np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2),
      #            'meps_westerly [mm/24h]:', 
       #           np.round(len(meps_westerly[int(year+month+day)][ens_memb]),2))

#            elif len(meps_easterly[int(year+month+day)][ens_memb]) == 0:  # only westerlies
#                print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 'EM=%s' %ens_memb,
  #                np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
   #                        np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2 ),
    #              'meps_easterly [mm/24h]:', 
     #             np.round(len(meps_easterly[int(year+month+day)][ens_memb]),2),
      #            'meps_westerly [mm/24h]:', 
       #           np.round(np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2))

#            else:    # both easterlies and westerlies
#                print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 'EM=%s' %ens_memb,
  #                np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
   #                        np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2 ),
    #              'meps_easterly [mm/24h]:', 
     #             np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2),
      #            'meps_westerly [mm/24h]:', 
       #           np.round(np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2))
            

            # assign sum values to days
            total = np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
                    np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb])
            meps_total_precip[int(year+month+day)].append(total)
    
            easterly = np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb])
            meps_precip_east[int(year+month+day)].append(easterly)
    
            westerly = np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb])
            meps_precip_west[int(year+month+day)].append(westerly)
            


no file found: ../../Data/MEPS/Haukeliseter/201612/sfc_hybrid5_allEM_00/20161216_00.nc
no file found: ../../Data/MEPS/Haukeliseter/201701/sfc_hybrid5_allEM_00/20170101_00.nc


### Retrieval data

In [8]:
ret_f = dict()
ret_accumulation = dict()
ret_total_precip = dict()
ret_precip_east = dict()
ret_precip_west = dict()


for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
        
        ret_dirnc = '../../Data/Retrieval/%s/combined/precip_wind_temp_%s%s%s.nc' %(station,year,month,day)
        ret_f[int(year+month+day)] = netCDF4.Dataset(ret_dirnc, 'r')

        ret_accumulation[int(year+month+day)] = (ret_f[int(year+month+day)].variables['precip_amount'][:])

        # assign sum values to days
        ret_total_precip[int(year+month+day)] = ret_f[int(year+month+day)].variables['24h_precip_sum'][:]
        ret_precip_east[int(year+month+day)] = ret_f[int(year+month+day)].variables['24h_upslope_sum'][:]
        ret_precip_west[int(year+month+day)] = ret_f[int(year+month+day)].variables['24h_pulsed_sum'][:]
        
        
#############################        
        ## set days, still with not exisiting observation data to 0 mm precipitation
        if np.isnan(obs_total_precip[int(year+month+day)]):
#            print([int(year+month+day)], 'total precip is NAN')
            obs_total_precip[int(year+month+day)] = 0.0
            meps_total_precip[int(year+month+day)] = np.zeros(shape=(10,))
            ret_total_precip[int(year+month+day)] = 0.0
        if np.isnan(obs_precip_east[int(year+month+day)]):
 #           print([int(year+month+day)], 'easterly is NAN')
            obs_precip_east[int(year+month+day)] = 0.0
            meps_precip_east[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_east[int(year+month+day)] = 0.0
        if np.isnan(obs_precip_west[int(year+month+day)]):
  #          print([int(year+month+day)], 'westerly is NAN')
            obs_precip_west[int(year+month+day)] = 0.0
            meps_precip_west[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_west[int(year+month+day)] = 0.0
        
        
        ## set days with less than 24h recorded values to 0mm precipitation 
        if np.count_nonzero(~np.isnan(obs_wind_dir_rolling[int(year+month+day)][:])) != 1440 and \
           np.count_nonzero(~np.isnan(obs_wind_dir_rolling[int(year+month+day)][:])) != 24:
   #         print([int(year+month+day)], 'less than 24h observation')
            obs_total_precip[int(year+month+day)] = 0.0
            meps_total_precip[int(year+month+day)] = np.zeros(shape=(10,))
            ret_total_precip[int(year+month+day)] = 0.0
            
            obs_precip_east[int(year+month+day)] = 0.0
            meps_precip_east[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_east[int(year+month+day)] = 0.0
            
            obs_precip_west[int(year+month+day)] = 0.0
            meps_precip_west[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_west[int(year+month+day)] = 0.0
            
            obs_total_precip_2deg[int(year+month+day)] = 0.0
            obs_precip_east_2deg[int(year+month+day)] = 0.0
            obs_precip_west_2deg[int(year+month+day)] = 0.0

        if obs_total_precip[int(year+month+day)]< 0.25:
    #        print([int(year+month+day)], 'less than 0.25 mm precipitation')
            obs_total_precip[int(year+month+day)] = 0.0
            meps_total_precip[int(year+month+day)] = np.zeros(shape=(10,))
            ret_total_precip[int(year+month+day)] = 0.0
            
            obs_precip_east[int(year+month+day)] = 0.0
            meps_precip_east[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_east[int(year+month+day)] = 0.0
            
            obs_precip_west[int(year+month+day)] = 0.0
            meps_precip_west[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_west[int(year+month+day)] = 0.0
            
            obs_total_precip_2deg[int(year+month+day)] = 0.0
            obs_precip_east_2deg[int(year+month+day)] = 0.0
            obs_precip_west_2deg[int(year+month+day)] = 0.0
            
#        print([int(year+month+day)], 'east', np.round(obs_precip_east_2deg[int(year+month+day)],2),
 #                                    'west', np.round(obs_precip_west_2deg[int(year+month+day)],2),
  #                                   'total', np.round(obs_total_precip_2deg[int(year+month+day)],2))
   #     
    #    print([int(year+month+day)], 'east', np.round(ret_precip_east[int(year+month+day)],2),
     #                                'west', np.round(ret_precip_west[int(year+month+day)],2),
      #                               'total', np.round(ret_total_precip[int(year+month+day)],2))
        
       

In [9]:
# calculate monthly precip sum observations for valid days
obs_total = []
obs_total_east = []
obs_total_west = []



for month in m:
    if month == '11':
#        t = np.arange(8,31)
        t = [9, 12, 13, 14, 15, 16, 17, 23, 24, 26,29,30]
    if month == '12': #or  or 
 #       t = np.arange(1,32)
        t = [7, 9, 10, 11, 12, 13, 15, 21, 22, 23, 24, 25, 26, 29, 30, 31]
    if month == '01':
        t = [2, 3, 6, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 28, 29]
    if month == '02':
  #      t = np.arange(1,29)
        t = [2, 3, 4, 6]
    if month == '03':
        t = [9, 14, 16, 17, 19, 21, 24, 30, 31]
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
     
    obs_monthly_sum = 0.0
    obs_monthly_east_sum = 0.0
    obs_monthly_west_sum = 0.0

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        tot_val = (obs_total_precip[int(year+month+day)])
        obs_monthly_sum = np.sum([obs_monthly_sum,tot_val])
        
        east_val = (obs_precip_east[int(year+month+day)])
        obs_monthly_east_sum = np.sum([obs_monthly_east_sum, east_val])
        
        west_val = (obs_precip_west[int(year+month+day)])
        obs_monthly_west_sum = np.sum([obs_monthly_west_sum, west_val])
        
        if int(day) == t[-1]:
            obs_total.append(obs_monthly_sum)
            obs_total_east.append(obs_monthly_east_sum)
            obs_total_west.append(obs_monthly_west_sum)
            
    print(int(year+month+day), 'total:', np.round(obs_monthly_sum,2),
                               'east:',  np.round(obs_monthly_east_sum,2),
                               'west:',  np.round(obs_monthly_west_sum),2)
#print('total:', np.round(sum(obs_total_precip.values()),2), 
 #     'east:', np.round(sum(obs_precip_east.values()),2), 
  #    'west:', np.round(sum(obs_precip_west.values()),2))

print('total:', np.round(np.sum(obs_total),2),
      'east:',  np.round(np.sum(obs_total_east), 2),
      'west:',  np.round(np.sum(obs_total_west), 2))

20161130 total: 46.76 east: 20.24 west: 27.0 2
20161231 total: 200.02 east: 23.23 west: 177.0 2
20170129 total: 90.1 east: 38.27 west: 52.0 2
20170206 total: 16.62 east: 16.25 west: 0.0 2
20170331 total: 92.31 east: 29.48 west: 63.0 2
total: 445.8 east: 127.48 west: 318.32


In [10]:
# calculate monthly precip sum observation for <2degC, and valid days
obs_total_2deg = []
obs_total_east_2deg = []
obs_total_west_2deg = []



for month in m:
    if month == '11':
#        t = np.arange(8,31)
        t = [9, 12, 13, 14, 15, 16, 17, 23, 24, 26,29,30]
    if month == '12': #or  or 
 #       t = np.arange(1,32)
        t = [7, 9, 10, 11, 12, 13, 15, 21, 22, 23, 24, 25, 26, 29, 30, 31]
    if month == '01':
        t = [2, 3, 6, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 28, 29]
    if month == '02':
  #      t = np.arange(1,29)
        t = [2, 3, 4, 6]
    if month == '03':
        t = [9, 14, 16, 17, 19, 21, 24, 30, 31]
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
      
    obs_monthly_sum_2deg = 0.0
    obs_monthly_east_sum_2deg = 0.0
    obs_monthly_west_sum_2deg = 0.0

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        tot_val_2deg = (obs_total_precip_2deg[int(year+month+day)])
        obs_monthly_sum_2deg = np.sum([obs_monthly_sum_2deg,tot_val_2deg])
        
        east_val_2deg = (obs_precip_east_2deg[int(year+month+day)])
        obs_monthly_east_sum_2deg = np.sum([obs_monthly_east_sum_2deg, east_val_2deg])
        
        west_val_2deg = (obs_precip_west_2deg[int(year+month+day)])
        obs_monthly_west_sum_2deg = np.sum([obs_monthly_west_sum_2deg, west_val_2deg])
        
        if int(day) == t[-1]:
            obs_total_2deg.append(obs_monthly_sum_2deg)
            obs_total_east_2deg.append(obs_monthly_east_sum_2deg)
            obs_total_west_2deg.append(obs_monthly_west_sum_2deg)
            
    print(int(year+month+day), 'total:', np.round(obs_monthly_sum_2deg,2),
                               'east:',  np.round(obs_monthly_east_sum_2deg,2),
                               'west:',  np.round(obs_monthly_west_sum_2deg),2)
#print('total:', np.round(sum(obs_total_precip.values()),2), 
 #     'east:', np.round(sum(obs_precip_east.values()),2), 
  #    'west:', np.round(sum(obs_precip_west.values()),2))

print('total:', np.round(np.sum(obs_total_2deg),2),
      'east:',  np.round(np.sum(obs_total_east_2deg), 2),
      'west:',  np.round(np.sum(obs_total_west_2deg), 2))

20161130 total: 42.5 east: 17.83 west: 25.0 2
20161231 total: 138.97 east: 22.45 west: 117.0 2
20170129 total: 86.65 east: 38.27 west: 48.0 2
20170206 total: 16.62 east: 16.25 west: 0.0 2
20170331 total: 83.95 east: 23.44 west: 61.0 2
total: 368.7 east: 118.25 west: 250.46


In [11]:
# calculate monthly precip sum for retrieved precipitation
ret_total = []
ret_total_east = []
ret_total_west = []


for month in m:
    if month == '11':
#        t = np.arange(8,31)
        t = [9, 12, 13, 14, 15, 16, 17, 23, 24, 26,29,30]
    if month == '12': #or  or 
 #       t = np.arange(1,32)
        t = [7, 9, 10, 11, 12, 13, 15, 21, 22, 23, 24, 25, 26, 29, 30, 31]
    if month == '01':
        t = [2, 3, 6, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 28, 29]
    if month == '02':
  #      t = np.arange(1,29)
        t = [2, 3, 4, 6]
    if month == '03':
        t = [9, 14, 16, 17, 19, 21, 24, 30, 31]
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    
    ret_monthly_sum = 0.0
    ret_monthly_east_sum = 0.0
    ret_monthly_west_sum = 0.0

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        tot_val = (ret_total_precip[int(year+month+day)])
        ret_monthly_sum = np.sum([ret_monthly_sum,tot_val])
        
        east_val = (ret_precip_east[int(year+month+day)])
        ret_monthly_east_sum = np.sum([ret_monthly_east_sum, east_val])
        
        west_val = (ret_precip_west[int(year+month+day)])
        ret_monthly_west_sum = np.sum([ret_monthly_west_sum, west_val])
        
        if int(day) == t[-1]:
            ret_total.append(ret_monthly_sum)
            ret_total_east.append(ret_monthly_east_sum)
            ret_total_west.append(ret_monthly_west_sum)
            
    print(int(year+month+day), 'total:', np.round(ret_monthly_sum,2),
                               'east:',  np.round(ret_monthly_east_sum,2),
                               'west:',  np.round(ret_monthly_west_sum,2))
#print('total:', np.round(sum(ret_total_precip.values()),2), 
 #     'east:', np.round(sum(ret_precip_east.values()),2), 
  #    'west:', np.round(sum(ret_precip_west.values()),2))

ret_sum = np.sum(ret_total,axis=0)
ret_east_sum = np.sum(ret_total_east, axis=0)
ret_west_sum = np.sum(ret_total_west, axis=0)

print('total:', np.round(np.sum(ret_total),2),
      'east:',  np.round(np.sum(ret_total_east), 2),
      'west:',  np.round(np.sum(ret_total_west), 2))

20161130 total: 42.63 east: 20.34 west: 22.29
20161231 total: 150.09 east: 22.94 west: 127.16
20170129 total: 89.9 east: 49.15 west: 40.75
20170206 total: 20.84 east: 20.74 west: 0.1
20170331 total: 90.4 east: 23.44 west: 66.96
total: 393.85 east: 136.61 west: 257.24


In [12]:
### retrieval seasonal statistics
diff_perc_monthly = np.zeros(shape=np.array(ret_total).shape)
diff_perc_season = np.zeros(shape=ret_sum.shape)

diff_perc_monthly_east = np.zeros(shape=np.array(ret_total_east).shape)
diff_perc_season_east = np.zeros(shape=ret_east_sum.shape)

diff_perc_monthly_west = np.zeros(shape=np.array(ret_total_west).shape)
diff_perc_season_west = np.zeros(shape=ret_west_sum.shape)

diff_perc_monthly = (ret_total - np.array(obs_total_2deg))/np.array(obs_total_2deg)*100
diff_perc_season  = (np.sum(ret_total) - np.sum(obs_total_2deg))/np.sum(obs_total_2deg)*100
    
diff_perc_monthly_east = (ret_total_east - np.array(obs_total_east_2deg))/np.array(obs_total_east_2deg)*100
diff_perc_season_east  = (ret_east_sum - np.sum(obs_total_east_2deg))/np.sum(obs_total_east_2deg)*100
    
diff_perc_monthly_west = (ret_total_west - np.array(obs_total_west_2deg))/np.array(obs_total_west_2deg)*100
diff_perc_season_west = (ret_west_sum - np.sum(obs_total_west_2deg))/np.sum(obs_total_west_2deg)*100

In [13]:
### create table

col_labels = ['obs [mm]', 'retrieved [mm]','Difference [%]', ]
row_labels = ['total precipitation', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017',
              '', 
              'pulsed - westerly', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017',
              '',
              'upslope - easterly', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017']
#for  in range(10):

table_vals = [ ['', '' , '' ],
               [np.round(obs_total_2deg[0],2), np.round(ret_total[0],2), np.round(diff_perc_monthly[0],2)],
               [np.round(obs_total_2deg[1],2), np.round(ret_total[1],2), np.round(diff_perc_monthly[1],2)],
               [np.round(obs_total_2deg[2],2), np.round(ret_total[2],2), np.round(diff_perc_monthly[2],2)],
               [np.round(obs_total_2deg[3],2), np.round(ret_total[3],2), np.round(diff_perc_monthly[3],2)],
               [np.round(obs_total_2deg[4],2), np.round(ret_total[4],2), np.round(diff_perc_monthly[4],2)],
               [np.round(np.sum(obs_total_2deg),2), np.round(ret_sum,2), np.round(diff_perc_season,2)],
               ['', '', ''],
               ['', '', '' ],
               [np.round(obs_total_west_2deg[0],2), np.round(ret_total_west[0],2), np.round(diff_perc_monthly_west[0],2)],
               [np.round(obs_total_west_2deg[1],2), np.round(ret_total_west[1],2), np.round(diff_perc_monthly_west[1],2)],
               [np.round(obs_total_west_2deg[2],2), np.round(ret_total_west[2],2), np.round(diff_perc_monthly_west[2],2)],
               [np.round(obs_total_west_2deg[3],2), np.round(ret_total_west[3],2), np.round(diff_perc_monthly_west[3],2)],
               [np.round(obs_total_west_2deg[4],2), np.round(ret_total_west[4],2), np.round(diff_perc_monthly_west[4],2)],
               [np.round(np.sum(obs_total_west_2deg),2), np.round(ret_west_sum,2), np.round(diff_perc_season_west,2)],
               ['', '', ''],
               ['', '', '' ],
               [np.round(obs_total_east_2deg[0],2), np.round(ret_total_east[0],2), np.round(diff_perc_monthly_east[0],2)],
               [np.round(obs_total_east_2deg[1],2), np.round(ret_total_east[1],2), np.round(diff_perc_monthly_east[1],2)],
               [np.round(obs_total_east_2deg[2],2), np.round(ret_total_east[2],2), np.round(diff_perc_monthly_east[2],2)],
               [np.round(obs_total_east_2deg[3],2), np.round(ret_total_east[3],2), np.round(diff_perc_monthly_east[3],2)],
               [np.round(obs_total_east_2deg[4],2), np.round(ret_total_east[4],2), np.round(diff_perc_monthly_east[4],2)],
               [np.round(np.sum(obs_total_east_2deg),2), np.round(ret_east_sum,2), np.round(diff_perc_season_east,2)]]


# Draw table
fig = plt.figure()
ax = fig.add_subplot(111)
the_table = plt.table(cellText = table_vals,
                     rowLabels = row_labels,
                     colLabels = col_labels,
                     loc = 'center'
                     )
the_table.auto_set_font_size(False)
the_table.set_fontsize(24)
the_table.scale(3,3.5)

# Removing ticks and spines enables you to get the figure only with table
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
    
yy = 2.83
fig_name = station+'_winter_16_17_retrieval'+'.'+form
ax.set_title( 'Winter 2016-2017 -- 24h accumulation', y =yy,
                 fontsize = 24)

if savefig == 1:
        plt.savefig('%s/%s' % (figdir, fig_name), format = form, bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir,fig_name))
else:
        plt.show()
plt.close()
    
    

plot saved: ../../Figures/Weathermast_MEPS_Retrieval/Haukeliseter/precip_seasonal_stat/Haukeliseter_winter_16_17_retrieval.png


In [ ]:
# calculate monthly precip sum for MEPS precipitation

meps_sum = 0.0
meps_east_sum = 0.0
meps_west_sum = 0.0
meps_total = []
meps_total_east = []
meps_total_west = []

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    
    for ens_memb in np.arange(0,10):
        meps_monthly_sum = 0.0
        meps_monthly_east_sum = 0.0
        meps_monthly_west_sum = 0.0
        
        
        
        for day in t:
            if day < 10:
                day = '0%s' %day
            else:
                day = '%s' %day
        
            tot_val = meps_total_precip[int(year+month+day)][ens_memb]
            meps_monthly_sum = np.nansum([meps_monthly_sum, tot_val])
        
            east_val = meps_precip_east[int(year+month+day)][ens_memb]
            meps_monthly_east_sum = np.nansum([meps_monthly_east_sum, east_val])
        
            west_val = meps_precip_west[int(year+month+day)][ens_memb]
            meps_monthly_west_sum = np.nansum([meps_monthly_west_sum, west_val])
        
            if int(day) == t[-1]:
                meps_total.append(meps_monthly_sum)
                meps_total_east.append(meps_monthly_east_sum)
                meps_total_west.append(meps_monthly_west_sum)

        print(int(year+month+day), 'EM = %s' %ens_memb, 
                               'total:', np.round(meps_monthly_sum,2),
                               'east:',  np.round(meps_monthly_east_sum,2),
                               'west:',  np.round(meps_monthly_west_sum,2))
        




meps_total = (np.reshape(meps_total, (5,10)))
meps_total_east = (np.reshape(meps_total_east, (5,10)))
meps_total_west = (np.reshape(meps_total_west, (5,10)))

meps_sum = np.sum(meps_total[:,:],axis=0)
meps_east_sum = np.sum(meps_total_east[:,:], axis=0)
meps_west_sum = np.sum(meps_total_west[:,:], axis=0)

for ens_memb in range(10):
    print('total:', 'EM = %s' %ens_memb, 
                               'total:', np.round(meps_sum[ens_memb],2),
                               'east:', np.round(meps_east_sum[ens_memb],2),
                               'west:', np.round(meps_west_sum[ens_memb],2))

In [ ]:
diff_perc_monthly = np.zeros(shape=meps_total.shape)
diff_perc_season = np.zeros(shape=meps_sum.shape)

diff_perc_monthly_east = np.zeros(shape=meps_total_east.shape)
diff_perc_season_east = np.zeros(shape=meps_east_sum.shape)

diff_perc_monthly_west = np.zeros(shape=meps_total_west.shape)
diff_perc_season_west = np.zeros(shape=meps_west_sum.shape)
for ens_memb in range(10):
    diff_perc_monthly[:,ens_memb] = (meps_total[:,ens_memb] - np.array(obs_total))/np.array(obs_total)*100
    diff_perc_season[ens_memb]  = (meps_sum[ens_memb] - sum(obs_total_precip.values()))/sum(obs_total_precip.values())*100
    
    diff_perc_monthly_east[:,ens_memb] = (meps_total_east[:,ens_memb] - np.array(obs_total_east))/np.array(obs_total_east)*100
    diff_perc_season_east[ens_memb]  = (meps_east_sum[ens_memb] - sum(obs_precip_east.values()))/sum(obs_precip_east.values())*100
    
    diff_perc_monthly_west[:,ens_memb] = (meps_total_west[:,ens_memb] - np.array(obs_total_west))/np.array(obs_total_west)*100
    diff_perc_season_west[ens_memb]  = (meps_west_sum[ens_memb] - sum(obs_precip_west.values()))/sum(obs_precip_west.values())*100

In [ ]:
### create table

col_labels = ['obs [mm]', 'MEPS [mm]','Difference [%]', ]
row_labels = ['total precipitation', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017',
              '', 
              'pulsed - westerly', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017',
              '',
              'upslope - easterly', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017']
#for ens_memb in range(10):
for ens_memb in np.arange(10):
    table_vals = [ ['', 'ensemble member %s' %ens_memb, '' ],
               [np.round(obs_total[0],2), np.round(meps_total[0,ens_memb],2), np.round(diff_perc_monthly[0,ens_memb],2)],
               [np.round(obs_total[1],2), np.round(meps_total[1,ens_memb],2), np.round(diff_perc_monthly[1,ens_memb],2)],
               [np.round(obs_total[2],2), np.round(meps_total[2,ens_memb],2), np.round(diff_perc_monthly[2,ens_memb],2)],
               [np.round(obs_total[3],2), np.round(meps_total[3,ens_memb],2), np.round(diff_perc_monthly[3,ens_memb],2)],
               [np.round(obs_total[4],2), np.round(meps_total[4,ens_memb],2), np.round(diff_perc_monthly[4,ens_memb],2)],
               [np.round(sum(obs_total_precip.values()),2), np.round(meps_sum[ens_memb],2), np.round(diff_perc_season[ens_memb],2)],
               ['', '', ''],
               ['', 'ensemble member %s' %ens_memb, '' ],
               [np.round(obs_total_west[0],2), np.round(meps_total_west[0,ens_memb],2), np.round(diff_perc_monthly_west[0,ens_memb],2)],
               [np.round(obs_total_west[1],2), np.round(meps_total_west[1,ens_memb],2), np.round(diff_perc_monthly_west[1,ens_memb],2)],
               [np.round(obs_total_west[2],2), np.round(meps_total_west[2,ens_memb],2), np.round(diff_perc_monthly_west[2,ens_memb],2)],
               [np.round(obs_total_west[3],2), np.round(meps_total_west[3,ens_memb],2), np.round(diff_perc_monthly_west[3,ens_memb],2)],
               [np.round(obs_total_west[4],2), np.round(meps_total_west[4,ens_memb],2), np.round(diff_perc_monthly_west[4,ens_memb],2)],
               [np.round(sum(obs_precip_west.values()),2), np.round(meps_west_sum[ens_memb],2), np.round(diff_perc_season_west[ens_memb],2)],
               ['', '', ''],
               ['', 'ensemble member %s' %ens_memb, '' ],
               [np.round(obs_total_east[0],2), np.round(meps_total_east[0,ens_memb],2), np.round(diff_perc_monthly_east[0,ens_memb],2)],
               [np.round(obs_total_east[1],2), np.round(meps_total_east[1,ens_memb],2), np.round(diff_perc_monthly_east[1,ens_memb],2)],
               [np.round(obs_total_east[2],2), np.round(meps_total_east[2,ens_memb],2), np.round(diff_perc_monthly_east[2,ens_memb],2)],
               [np.round(obs_total_east[3],2), np.round(meps_total_east[3,ens_memb],2), np.round(diff_perc_monthly_east[3,ens_memb],2)],
               [np.round(obs_total_east[4],2), np.round(meps_total_east[4,ens_memb],2), np.round(diff_perc_monthly_east[4,ens_memb],2)],
               [np.round(sum(obs_precip_east.values()),2), np.round(meps_east_sum[ens_memb],2), np.round(diff_perc_season_east[ens_memb],2)]]


# Draw table
    fig = plt.figure()
    ax = fig.add_subplot(111)
    the_table = plt.table(cellText = table_vals,
                     rowLabels = row_labels,
                     colLabels = col_labels,
                     loc = 'center'
                     )
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(24)
    the_table.scale(3,3.5)

# Removing ticks and spines enables you to get the figure only with table
    plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
    for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
    
    yy = 2.83
    fig_name = station+'_winter_16_17_EM'+str(ens_memb)+'.'+form
    ax.set_title( 'Winter 2016-2017 -- 24h accumulation', y =yy,
                 fontsize = 24)

    if savefig == 1:
        plt.savefig('%s/%s' % (figdir, fig_name), format = form, bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir,fig_name))
    else:
        plt.show()
    plt.close()
    
    if ens_memb == 9:
        table_vals = [ ['', 'ensemble mean' , '' ],
               [np.round(obs_total[0],2), np.round(np.mean(meps_total, axis = 1)[0],2), np.round(np.mean(diff_perc_monthly, axis=1)[0],2)],
               [np.round(obs_total[1],2), np.round(np.mean(meps_total, axis = 1)[1],2), np.round(np.mean(diff_perc_monthly, axis=1)[1],2)],
               [np.round(obs_total[2],2), np.round(np.mean(meps_total, axis = 1)[2],2), np.round(np.mean(diff_perc_monthly, axis=1)[2],2)],
               [np.round(obs_total[3],2), np.round(np.mean(meps_total, axis = 1)[3],2), np.round(np.mean(diff_perc_monthly, axis=1)[3],2)],
               [np.round(obs_total[4],2), np.round(np.mean(meps_total, axis = 1)[4],2), np.round(np.mean(diff_perc_monthly, axis=1)[4],2)],
               [np.round(sum(obs_total_precip.values()),2), np.round(np.mean(meps_sum, axis = 0),2), np.round(np.mean(diff_perc_season, axis = 0),2)],
               ['', '', ''],
               ['', 'ensemble mean' , '' ],
               [np.round(obs_total_west[0],2), np.round(np.mean(meps_total_west, axis=1)[0],2), np.round(np.mean(diff_perc_monthly_west, axis=1)[0],2)],
               [np.round(obs_total_west[1],2), np.round(np.mean(meps_total_west, axis=1)[1],2), np.round(np.mean(diff_perc_monthly_west, axis=1)[1],2)],
               [np.round(obs_total_west[2],2), np.round(np.mean(meps_total_west, axis=1)[2],2), np.round(np.mean(diff_perc_monthly_west, axis=1)[2],2)],
               [np.round(obs_total_west[3],2), np.round(np.mean(meps_total_west, axis=1)[3],2), np.round(np.mean(diff_perc_monthly_west, axis=1)[3],2)],
               [np.round(obs_total_west[4],2), np.round(np.mean(meps_total_west, axis=1)[4],2), np.round(np.mean(diff_perc_monthly_west, axis=1)[4],2)],
               [np.round(sum(obs_precip_west.values()),2), np.round(np.mean(meps_west_sum, axis = 0),2), np.round(np.mean(diff_perc_season_west, axis = 0),2)],
               ['', '', ''],
               ['', 'ensemble mean' , '' ],
               [np.round(obs_total_east[0],2), np.round(np.mean(meps_total_east, axis=1)[0],2), np.round(np.mean(diff_perc_monthly_east, axis=1)[0],2)],
               [np.round(obs_total_east[1],2), np.round(np.mean(meps_total_east, axis=1)[1],2), np.round(np.mean(diff_perc_monthly_east, axis=1)[1],2)],
               [np.round(obs_total_east[2],2), np.round(np.mean(meps_total_east, axis=1)[2],2), np.round(np.mean(diff_perc_monthly_east, axis=1)[2],2)],
               [np.round(obs_total_east[3],2), np.round(np.mean(meps_total_east, axis=1)[3],2), np.round(np.mean(diff_perc_monthly_east, axis=1)[3],2)],
               [np.round(obs_total_east[4],2), np.round(np.mean(meps_total_east, axis=1)[4],2), np.round(np.mean(diff_perc_monthly_east, axis=1)[4],2)],
               [np.round(sum(obs_precip_east.values()),2), np.round(np.mean(meps_east_sum, axis = 0),2), np.round(np.mean(diff_perc_season_east, axis = 0),2)]]
        # Draw table
        fig = plt.figure()
        ax = fig.add_subplot(111)
        the_table = plt.table(cellText = table_vals,
                     rowLabels = row_labels,
                     colLabels = col_labels,
                     loc = 'center'
                     )
        the_table.auto_set_font_size(False)
        the_table.set_fontsize(24)
        the_table.scale(3,3.5)

# Removing ticks and spines enables you to get the figure only with table
        plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
        plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
        for pos in ['right','top','bottom','left']:
            plt.gca().spines[pos].set_visible(False)
        yy = 2.83
        fig_name = station+'_winter_16_17_ensemble_mean'+'.'+form
        ax.set_title( 'Winter 2016-2017 -- 24h accumulation', y =yy,
                 fontsize = 24)

        if savefig == 1:
            plt.savefig('%s/%s' % (figdir, fig_name), format = form, bbox_inches='tight')
            print('plot saved: %s/%s' %(figdir,fig_name))
        else:
            plt.show()
        plt.close()